# モデルのテスト

作成したモデルをもとに、新しいデータでテストをしていきましょう。

新しいデータはBank_data_testing.csvにあります。


## ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

## データの読み込み

'Bank_data.csv'データを読み込みます

In [ ]:
raw_data = pd.read_csv('Bank-data.csv')
raw_data

We want to know whether the bank marketing strategy was successful, so we need to transform the outcome variable into Boolean values in order to run regressions.

In [ ]:
# 読み込んだデータを新しい変数に代入します
data = raw_data.copy()
# インデックスの列を削除します
data = data.drop(['Unnamed: 0'], axis = 1)
# yesとnoをと0に変換するためにmapメソッドを用いていきます 
data['y'] = data['y'].map({'yes':1, 'no':0})
data

In [ ]:
data.describe()

### 従属変数と独立変数の宣言

独立変数として 'duration' を使います

In [ ]:
y = data['y']
x1 = data['duration']

### ロジスティック回帰

回帰を行い、散布図を表示します

In [ ]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
# 回帰のまとめ表
results_log.summary()

In [ ]:
# x1とyの散布図の作成
plt.scatter(x1,y,color = 'C0')
# 軸に名前を付ける
plt.xlabel('Duration', fontsize = 20)
plt.ylabel('Subscription', fontsize = 20)
plt.show()

## モデルの拡張

‘interest_rate’, ‘march’, ‘credit’ , ‘previous’を追加して再度回帰モデルを作成していきます

### 独立変数の宣言

In [ ]:
# 追加する特徴量のリストを作成します 
estimators=['interest_rate','credit','march','previous','duration']

X1_all = data[estimators]
y = data['y']

In [ ]:
X_all = sm.add_constant(X1_all)
reg_logit = sm.Logit(y,X_all)
results_logit = reg_logit.fit()
results_logit.summary2()

### 混同行列

混同行列を作成します

In [ ]:
def confusion_matrix(data,actual_values,model):
        #Logitモデルを使って値を予測します
        pred_values = model.predict(data)
        # 階級を指定します
        bins=np.array([0,0.5,1])
        # ヒストグラムを作成します。ここで、値が0 と 0.5 の間であれば値は0とします
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # 正確性を計算します
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # 混同行列と正確性を返します 
        return cm, accuracy

In [ ]:
confusion_matrix(X_all,y,results_logit)

## モデルのテスト

 'Bank_data_testing.csv' からテストデータを抽出します

### データの読み込み 

In [ ]:
# テストデータを読み込みます
raw_data2 = pd.read_csv('Bank-data-testing.csv')
data_test = raw_data2.copy()
# インデックスの列を削除します
data_test = data_test.drop(['Unnamed: 0'], axis = 1)

In [ ]:
# yesとnoを1と0に変換します
data_test['y'] = data_test['y'].map({'yes':1, 'no':0})
data_test

### 従属変数と独立変数の宣言

In [ ]:
y_test = data_test['y']
X1_test = data_test[estimators]
X_test = sm.add_constant(X1_test)

訓練用の結果と比較するために、テスト用データに基づく混同行列と正確性を求めます

In [ ]:
confusion_matrix(X_test, y_test, results_logit)

In [ ]:
confusion_matrix(X_all,y, results_logit)

テストデータによる結果が86.04%で、訓練用データによる結果が86.29%となりました。

一般的には、訓練用データの方が正確性が高くなることが多いです。